<a href="https://colab.research.google.com/github/Niel-77/Geminiagent/blob/main/Two_Agent_team_for_MechE_problems_using_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creating two agent team using Gemini
This is the notebook for creating conversable agent using gemini api. In this notebook we will create a conversable agent to solve a simple mechanical engineering problem. We will be usingpython package, FEniCS65 as the simulation environment for solving elasticity problems using FEM.


## Creating Gemini API key
We	can obtain Gemini API key from https://aistudio.google.com/. Gemini 1.0, Gemini 1.5 Pro and Gemini 1.5 Flash can be used for free of charge but with limits set to each of them for Requests per minutes (RPM), tokens per minute (TPM) and requests per day (RPD).

## Preparing FeniCS for Colab
We will install the fenics package using the github link if it not installed already.

In [ ]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

--2024-06-05 09:30:47--  https://fem-on-colab.github.io/releases/fenics-install-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.111.153, 185.199.108.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4161 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.06K  --.-KB/s    in 0s      

2024-06-05 09:30:47 (43.1 MB/s) - ‘/tmp/fenics-install.sh’ saved [4161/4161]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/9669fbe/releases/pybind11-inst

## Prepare autogen for collab

In [ ]:
try:
    import chromadb
except ImportError:
    !pip install chromadb==0.4.15
    import chromadb

try:
    import tiktoken
except ImportError:
    !pip install tiktoken==0.5.1
    import tiktoken

try:
    import pypdf
except ImportError:
    !pip install pypdf==3.17.0
    import pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 

## Install Autogen with gemini features

In [ ]:
pip install pyautogen[gemini]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00


## Install the necessary dependencies

In [ ]:

# Install pyautogen with the required extras
!pip install pyautogen[gemini,retrievechat,lmm] --use-deprecated=legacy-resolver


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_6

## Importing the necessary functions
(All the imported functions might not have been used)

In [ ]:
import os
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import chromadb
from PIL import Image
from termcolor import colored

import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.img_utils import _to_pil, get_image_data
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

## Creating configuration for gemini and setting API endpoint
The similar configuration model can be used for all LLM models. For using gpt along with gemini, for example, we can add another list of called *config_list_gpt* with the model name and api key. This might not be the best way to create configuration when using multiple agent team but this will help us to avoid OAI_CONFIG_LIST which I found to be a little bit confusing to use with colab.

In [ ]:
from google.colab import userdata
geminiapi=userdata.get('GEMINIAPI')
config_list_gemini = [
     {
        "model": "gemini-pro",
        "api_key": geminiapi,
        "api_type": "google"
    }
]


In [ ]:
gemini_config = {
    "seed": 25,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gemini,
    "request_timeout": 600,
    "retry_wait_time": 120,
}

## Creating assistant called engineer
Here we create a assistant called engineer with system message telling that they are a professional mechanical engineer with extensive background in python coding and FeniCS who specialize at creating error free python codes.

In [ ]:
assistant = AssistantAgent(
    name="engineer",
    system_message="""
    You are a mechanical engineer with extensive background in python and FeniCS .\
    You are a specialist in creating error free codes.\
    You write code such that even an amateur can understand it easily.\
    You say 'DONE' when you are done.""",
    llm_config=gemini_config,
    max_consecutive_auto_reply=3)

user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config={"work_dir": "Mechanics", "use_docker": False},
    human_input_mode="NEVER",
    is_termination_msg=lambda x: content_str(x.get("content")).find("TERMINATE") >= 0,
)

## Provide question to the engineer
The engineer receives the task description from the user.

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="""A 1m-by-1m elastic plate has Young's modulus of 1GPa and Poisson ration of 0.3.
    It has zero displacement on the left edge and 0.1m displacement along x direction on the right edge.
    Solve for the displacement using FENICS and store the displacement result in a PNG file.""",
)

user_proxy (to engineer):

A 1m-by-1m elastic plate has Young's modulus of 1GPa and Poisson ration of 0.3.
    It has zero displacement on the left edge and 0.1m displacement along x direction on the right edge.
    Solve for the displacement using FENICS and store the displacement result in a PNG file.

--------------------------------------------------------------------------------
engineer (to user_proxy):

```python
import dolfin as df
import matplotlib.pyplot as plt

# Define the computational domain
mesh = df.UnitSquareMesh(10, 10)

# Define the function space
V = df.VectorFunctionSpace(mesh, "Lagrange", 1)

# Define the boundary conditions
bc = df.DirichletBC(V, df.Constant((0.0, 0.0)), "on_boundary")

# Define the variational problem
u = df.TrialFunction(V)
v = df.TestFunction(V)
a = df.inner(df.grad(u), df.grad(v)) * df.dx
L = df.inner(df.Constant((0.0, 0.1)), v) * df.dx

# Solve the variational problem
u = df.Function(V)
solve(a == L, u, bc)

# Plot the solution
plt.figure()


ChatResult(chat_id=None, chat_history=[{'content': "A 1m-by-1m elastic plate has Young's modulus of 1GPa and Poisson ration of 0.3.\n    It has zero displacement on the left edge and 0.1m displacement along x direction on the right edge.\n    Solve for the displacement using FENICS and store the displacement result in a PNG file.", 'role': 'assistant'}, {'content': '```python\nimport dolfin as df\nimport matplotlib.pyplot as plt\n\n# Define the computational domain\nmesh = df.UnitSquareMesh(10, 10)\n\n# Define the function space\nV = df.VectorFunctionSpace(mesh, "Lagrange", 1)\n\n# Define the boundary conditions\nbc = df.DirichletBC(V, df.Constant((0.0, 0.0)), "on_boundary")\n\n# Define the variational problem\nu = df.TrialFunction(V)\nv = df.TestFunction(V)\na = df.inner(df.grad(u), df.grad(v)) * df.dx\nL = df.inner(df.Constant((0.0, 0.1)), v) * df.dx\n\n# Solve the variational problem\nu = df.Function(V)\nsolve(a == L, u, bc)\n\n# Plot the solution\nplt.figure()\ndf.plot(u)\nplt.show